In [20]:
import pandas as pd
import numpy as np
import subprocess as sp
from rpy2.robjects import r, pandas2ri

pandas2ri.activate()

In [2]:
popseq = pd.read_pickle("180130_wheat_CSS_POPSEQ.pickle.gz")

In [33]:
! ls -lrth *pickle*

-rw-rw-rw- 1 lucve lucve  12M May 12 19:52 180130_wheat_CSS_POPSEQ.pickle.gz
-rw-rw-rw- 1 lucve lucve 3.4M May 12 20:17 assembly.initial_assembly.pickle.gz
-rw-rw-rw- 1 lucve lucve  27M May 12 20:18 assembly.anchored_assembly.pickle
-rw-rw-rw- 1 lucve lucve  967 May 12 20:18 assembly.initial_breaks.pickle.gz


In [34]:
! ls -lrth Minimal/*Rds

-rw-rw-rw- 1 lucve lucve 3.5M May 12 20:22 Minimal/assembly_init.Rds
-rw-rw-rw- 1 lucve lucve  14M May 12 20:22 Minimal/alchemy_v0.Rds
-rw-rw-rw- 1 lucve lucve  603 May 12 20:22 Minimal/alchemy_v0_breaks.Rds
-rw-rw-rw- 1 lucve lucve  14M May 12 20:23 Minimal/20200414_assembly_v1.Rds
-rw-rw-rw- 1 lucve lucve  815 May 12 20:23 Minimal/20200414_assembly_breaks.Rds


In [31]:
# These are now identical

r_initial_r = r["readRDS"]("Minimal/assembly_init.Rds")
r_initial = dict(zip(r_initial_r.names, r_initial_r))

initial = pd.read_pickle("assembly.initial_assembly.pickle.gz")

ValueError: Can only compare identically-labeled DataFrame objects

In [36]:
r_anchored_r = r["readRDS"]("Minimal/alchemy_v0.Rds")
r_anchored = dict(zip(r_anchored_r.names, r_anchored_r))

In [37]:
! mv assembly.anchored_assembly.pickle assembly.anchored_assembly.pickle.gz

In [60]:
anchored = pd.read_pickle("assembly.anchored_assembly.pickle.gz")

In [61]:
for key in r_anchored:
    if isinstance(r_anchored[key], pd.DataFrame):
        print(key, r_anchored[key].shape, anchored[key].shape)
    else:
        print(key, r_anchored[key], anchored[key])

info (100, 46) (100, 46)
cssaln (8568, 16) (8568, 16)
fpairs (4818, 10) (4818, 10)
molecules (130892, 10) (130892, 10)
popseq (1552095, 10) (1552095, 10)
molecule_cov (200421, 8) (200421, 8)
mol_binsize [200.] 200
cov (6830, 10) (6830, 10)
binsize [5000.] 5000.0
minNbin [50.] 50
innerDist [300000.] 300000.0


In [59]:
r_initial["fpairs"].shape

(4818, 8)

In [63]:
r_v1_r = r["readRDS"]("Minimal/20200414_assembly_v1.Rds")
r_v1 = dict(zip(r_v1.names, r_v1))

In [262]:
v1 = pd.read_pickle("assembly.assembly_v1.pickle.gz")

In [75]:
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

fai = r["readRDS"]("Minimal/test_fai.Rds")
assembly = r["readRDS"]("Minimal/assembly_init.Rds")
assembly = dict(zip(assembly.names, assembly))
fpairs = assembly["fpairs"]

In [263]:
v1.keys()

dict_keys(['info', 'cssaln', 'fpairs', 'molecules', 'popseq', 'mol_binsize', 'molecule_cov', 'cov', 'binsize', 'innerDist', 'minNbin'])

In [264]:
r_v1.keys()

dict_keys(['info', 'cssaln', 'fpairs', 'molecules', 'breaks', 'popseq', 'cov', 'mol_binsize', 'molecule_cov', 'binsize', 'innerDist', 'minNbin'])

In [267]:
for key in r_v1:
    if key not in v1:
        v1_val = None
    else:
        v1_val = v1[key]
        if isinstance(v1_val, pd.DataFrame):
            v1_val = v1_val.shape
    if isinstance(r_v1[key], pd.DataFrame):
        print(key, r_v1[key].shape, v1_val)
    else:
        print(key, r_v1[key], v1_val)

info (126, 47) (122, 47)
cssaln (8568, 16) (8568, 16)
fpairs (4818, 10) (4818, 10)
molecules (130808, 10) (126047, 10)
breaks (2, 10) None
popseq (1552095, 10) (1552095, 10)
cov (6816, 10) (6830, 10)
mol_binsize [200.] 200
molecule_cov (200337, 8) (569370, 8)
binsize [5000.] 5000.0
innerDist [300000.] 300000.0
minNbin [50.] 50


In [274]:
r_v1["info"].orig_scaffold.unique().shape

(100,)

In [273]:
v1["info"].orig_scaffold.unique().shape

(100,)

In [284]:
min_npairs = 2
max_dist = 1e5

info = v1["info"]
z = v1["molecules"].loc[v1["molecules"].eval("npairs >= {min_npairs}".format(**locals())), :]
z = info.loc[:, ["scaffold", "length"]].rename(columns={"length": "scaffold_length"}).merge(
        z, on="scaffold", how="right")
z = z.loc[(z["end"] <= max_dist) | (z["scaffold_length"] - z["start"] <= max_dist), :]
z.loc[:, "nsc"] = z.groupby(["barcode", "sample"])["scaffold"].transform(
        lambda series: series.unique().shape[0])

In [287]:
_z = z.copy()
z = z.loc[z["nsc"] >= 2]

In [295]:
x = z.loc[:, ["scaffold", "npairs", "sample", "barcode"]].rename(
        columns={"scaffold": "scaffold1", "npairs": "npairs1"}).assign(pos1=((z["start"] + z["end"])/2).astype(int))[
        ["scaffold1", "npairs1", "pos1", "sample", "barcode"]]
y = z.loc[:, ["scaffold", "npairs", "sample", "barcode"]].rename(
        columns={"scaffold": "scaffold2", "npairs": "npairs2"}).assign(pos2=((z["start"] + z["end"])/2).astype(int))[
        ["scaffold2", "npairs2", "pos2", "sample", "barcode"]]

In [300]:
print(y.shape)
print(x.shape)
xy = y.merge(x, on=["sample", "barcode"], how="outer").loc[lambda df: df.eval("scaffold1 != scaffold2")]
link_pos = xy.copy()

(214, 5)
(214, 5)


In [308]:
w = xy.groupby(["scaffold1", "scaffold2", "sample"]).size().to_frame("nmol").reset_index(drop=False)
min_nmol = 1
ww = w.loc[w["nmol"] >= min_nmol, :]
min_nsample = 2
ww.loc[ww.groupby(["scaffold1", "scaffold2"])["sample"].transform(lambda series: series.unique().shape[0]) >= min_nsample, :]

,scaffold1,scaffold2,sample,nmol
1,scaffold_corrected_000001,scaffold_corrected_000598,Sample_PRO1373_S20_LIB35277_190111,1
2,scaffold_corrected_000001,scaffold_corrected_000598,Sample_PRO1373_S20_LIB35277_190306,1
5,scaffold_corrected_000020,scaffold_corrected_000324,Sample_PRO1373_S20_LIB35275_190111,1
6,scaffold_corrected_000020,scaffold_corrected_000324,Sample_PRO1373_S20_LIB35275_190306,1
8,scaffold_corrected_000026,scaffold_corrected_000382,Sample_PRO1373_S20_LIB35275_190111,1
...,...,...,...,...
196,scaffold_corrected_000946,scaffold_corrected_000924,Sample_PRO1373_S20_LIB35275_190306,1
197,scaffold_corrected_000960,scaffold_corrected_000544,Sample_PRO1373_S20_LIB35277_190111,1
198,scaffold_corrected_000960,scaffold_corrected_000544,Sample_PRO1373_S20_LIB35277_190306,1
210,scaffold_corrected_991,scaffold_corrected_000492,Sample_PRO1373_S20_LIB35275_190111,1


In [ ]:
def rolling_join(left: pd.DataFrame, right: pd.DataFrame, on, by):
        
    idx_names = left.index.names
    left_grouped = left.sort_values(by).resegroupby(on)
    right.loc[:, "left_index"] = right.apply(
        lambda row: np.searchsorted(left_grouped.get_group(row[on])[by], row[by]), axis=1)
    
    